In [1]:
import pandas as pd
import numpy as np
# from operator import itemgetter
from sentence_transformers import SentenceTransformer

In [2]:
df = pd.read_csv('results/data.gov.lt.csv')[:200]
df['id'] = df.index
corpus = list(df['Name of the dataset'].values)

In [3]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(corpus)

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(
    [sentence_embeddings[0]],
    sentence_embeddings[1:10]
)


array([[0.90425277, 0.88191164, 0.43846297, 0.4122041 , 0.30720556,
        0.40834227, 0.46912238, 0.4781108 , 0.49695876]], dtype=float32)

In [ ]:
def get_top(X,rank):
    cosines = {}
    cosines_n = {}
    for x in range(X.shape[0]):
        cossims = [cosine_similarity(x, X)]
        # print('I', x)
        d = {}
        dn = {}
        for y in range(X.shape[0]):
            if x != y:
                cossim = cosine_similarity(X[x],X[y])
                d[corpus[y]] = cossim
                dn[y] = cossim
        # d = {corpus[y]:cosine_similarity(X[x],X[y])[0][0] for y in range (X.shape[0]) if x != y}
        cosdict = dict(sorted(d.items(), key = itemgetter(1), reverse = True)[:rank])
        cos_n_dict = dict(sorted(dn.items(), key = itemgetter(1), reverse = True)[:rank])
        cosines[x] =  '\n'.join([str(k) + ':'+ str(v) for k,v in cosdict.items()])
        cosines_n[x] =  ';'.join([str(k) + ':'+ str(v) for k,v in cos_n_dict.items()])
        print(corpus[x], cosines[x])
    return cosines, cosines_n

In [ ]:
from operator import itemgetter
sdict = {}
for x in range(sentence_embeddings.shape[0]):
        cossims = cosine_similarity([sentence_embeddings[x]],sentence_embeddings)[0]
        d = dict(zip(corpus,cossims))
        d.pop(corpus[x])
        cosdict = dict(sorted(d.items(), key = itemgetter(1), reverse = True)[:5])
        # cossims = [cosine_similarity(sentence_embeddings[x], sentence_embeddings)]
        print(corpus[x])
        print(cosdict)
        sdict[x] = '\n'.join([str(k) + ':'+ str(v) for k,v in cosdict.items()])
print(sdict)
print(len(sdict))


In [38]:
df['similar'] = df['id'].map(sdict)


In [39]:
df.to_csv('results/data.gov.lt-simbert.csv')